# Get List of Items & Quantities from All Orders

In [1]:
import pandas as pd
import re
from pathlib import Path
import os

In [2]:
OUTPUT_DIR = Path("C:/Playground/bhaiya-orders/outputs/")

In [3]:
from ipyfilechooser import FileChooser

In [4]:
fc = FileChooser("C:/Playground/bhaiya-orders/")
fc.use_dir_icons = True
display(fc)

FileChooser(path='C:/Playground/bhaiya-orders/', filename='', show_hidden='False')

In [5]:
FILE = Path(fc.selected)

Try to read valid `csv` and `xlsx` files into a dataframe `df`. 

In [6]:
try:
    if 'csv' in FILE.suffix: df = pd.read_csv(FILE)
    else: df = pd.read_excel(FILE)
except OSError:
    print("Error occured, invalid file!")

In [7]:
df.head(2)

,Submission Date,Order Number#,First Name,Last Name,Select Drop Point and Building,Any other building?,Select Wing,Flat No,Phone Number,Email,Select Delivery Date,Your Order: Products,No Label,Any pending unadjusted credit from previous orders?
0,2020-07-22 21:52:53,BHOR-0976,Deepika,Mehta,NCP - Tower 6,NaN,B Wing,3302 b wing,9654138008,deepikamehta.16@gmail.com,Friday 24th July,"Potato 1 Kg (Amount: 80.00 INR, Special Quanti...",Accepted\n\n1. All issues and complaints regar...,NaN
1,2020-07-22 21:03:50,BHOR-0975,Nisha,Narale,Cosmic - Cosmic Heights,NaN,A Wing,2505,9930390994,nishashetti@gmail.com,Friday 24th July,"Baby Potato 500 Gms (Amount: 20.00 INR, Specia...",Accepted\n\n1. All issues and complaints regar...,NaN


## Identify the Relevant Columns

Pick out the relevant column for orders.

**Assumptions:** 
1. If a column header contains the substring `"Your Order"`, it is deemed to be the required column. Take care that no other colummn contains the substring.
2. "Token": Token Number
3. "Flat": Flat Number

In [8]:
for col in df.columns:
    if "Your Order" in col:
        your_order = col
        
    if "token" in col.lower():
        token = col
        
    if "flat" in col.lower():
        flat = col

## Prepare Keys

Pick out all the possible unique orders from the given spreadsheet.

**Assumptions**: 
1. Every order for a particular user is separated by a closing paranthesis `)`.
2. The key-value pair in an item is ordered as `key (value)`. 
3. The last item in a list of orders is always the column describing total cost.

In [9]:
k = []
for order in df[your_order]:
    split = order.split(')')[:-1] # last item is always total cost
    split = [s.replace('\n', '') for s in split] # remove newlines if present
    k.append(split)

In [10]:
def get_qty(txt):
    
    """
        Returns the Special Quantity key for a given order. 
            Sample input string: '...no. of 500 gm packets required: 1)'
            Returns: 1
    """
    
    match = re.search(": ([0-9]+?)\)", txt)
    if match:
        return int(match.group(1))

def get_price(txt):
    
    """
        Returns the Special Quantity key for a given order. 
            Sample input string: '...no. of 500 gm packets required: 1)'
            Returns: 1
    """
    
    match = re.search("Amount: (.*?) INR", txt)
    if match:
        return (match.group(1))

In [11]:
all_costs = {}

In [12]:
items = []
for order in k:
    for item in order:
        if len(item.split('(')) > 1: 
            items.append(item.split('(')[0].strip())
            all_costs[item.split('(')[0].strip()] = float(get_price(item).replace(',', '')) / get_qty(item+')')

In [13]:
all_items = sorted(list(set(items)))

In [14]:
all_items

['Apples Royal Gala Per Kg',
 'Apples Washington Per Kg',
 'Arbi Leaves Pack of 5 leaves',
 'Arbi/ Colocasia 500gms',
 'Baby Corn 300 gms',
 'Baby Potato 500 Gms',
 'Banana Elaichi Per Dozen',
 'Banana Regular Per 6 Pcs',
 'Basil Leaves  200 gms',
 'Beetroot 500 Gms',
 "Bhindi/Lady's Finger 500 Gms",
 'Bitter Gourd 300 Gms',
 'Bottle Gourd Per Pc',
 'Brinjal Bharta Per Pc',
 'Brinjal Kateri 500 Gms',
 'Broccoli  250 gms',
 'Button Mushroom 200 gms Pack',
 'Cabbage Per Pc',
 'Capsicum Green 500 gms',
 'Cauliflower Per Pc',
 'Celery 200 Gms',
 'Cherry Tomato 250 Gms',
 'Coconut Mangalore Big Per Pc 525 gm',
 'Coconut Mangalore Medium 425 gms',
 'Coconut Mangalore Small per 300 gms',
 'Coriander Leaves per bunch 250 - 300 gms',
 'Cucumber 500 Gms',
 'Curry Leaves per bunch',
 'Garlic 250 Gms',
 'Gawar/Cluster Beans 300 gms',
 'Ginger 250 Gms',
 'Green Chilly 200 Gms',
 'Green Zucchini Per Pc',
 'Kiwi Imported Pack of 3 Pc',
 'Lemongrass  per bunch',
 'Lemons pack of 5',
 'Lettuce 250 gms'

## Prepare Count Dictionary

Count total orders for a particular item in the given spreadsheet.

**Assumptions**:

1. The value count for a particular order is sandwiched as an `integer` as `: INT)`. For example, _'Potato 1 Kg (Amount: 36.00 INR, Special Quantity: 10)'_

In [15]:
# init order dictionary
all_orders = {}
for item in all_items:
    all_orders[item] = 0

In [16]:
for order in k:
    for i in order:
        i = i + ')'
        sp1 = i.split("(")
        if len(sp1) < 2:
            continue
        
        key = sp1[0].strip()
        val = get_qty(sp1[1])
        if(key[:3] == "VNR"): val = 1
#         if(len(key) < 3): print(order)
        all_orders[key] += val

## Create DataFrame

In [17]:
# create empty dataframe
cleaned_df = pd.DataFrame.from_dict(all_orders, orient="index")
cleaned_df.reset_index(inplace=True)
cleaned_df.columns = ["item", "num_units"]

### Quantity Per Packet

Pull out the quantity in each order.

**Assumptions**:
1. If integers are present in the key, then the last integer value is chosen as the packet size for that particular order. For example, *'Tondli 300-400 gms'* returns a packet size of 400 (unit is extracted later).

In [18]:
def extract_num(txt):
    
    """
        Searches a string and returns the last number found, if present. 
        Otherwise return 1.
        Use case: For an item key such as 'Tondli 300 gms ', return 300 (to calculate total qty required)
    """
    
    match = re.findall("([0-9]+)", txt)
    if match:
        return int(match[-1])
    else:
        return 1

In [19]:
# find quantity per item (amount of stuff in one packet)
packet = []
for item in cleaned_df.item:
    packet.append(extract_num(item))
cleaned_df["qty_per_packet"] = packet

### Total Quantity to be Purchased

Self explanatory. `(Total quantity to be purchased) = (Size of one packet) * (Total number of packets ordered)`

In [20]:
# total quantity = #units ordered * qty in 1 unit
cleaned_df["total_qty"] = cleaned_df.num_units * cleaned_df.qty_per_packet

### Unit of Measurement

Extract unit of measurement for particular item.

**Approach**:
1. If ` gm` is in item: grams
2. Else, if ` pc` is in item: pieces
3. Else, if ` bunch` is in item: bunches
4. Default to kilograms

In [21]:
def get_unit(txt):
    """
        Gets unit from an item string.
        Assumption: standard occurence of units all across the board. 
    """
    txt = txt.lower()
    if txt.find('gm') != -1: unit = 'gms'
    elif txt.find(' pc') != -1: unit = 'pcs'
    elif txt.find(' bunch') != -1: unit = 'bunches'
    else: unit = 'kgs'
    return unit

In [22]:
# extract qty unit from item
cleaned_df["unit"] = cleaned_df.item.apply(get_unit)

### Handling the Curious Case of Bell Peppers

The key `'Red & Yellow Bell Pepper 2 each in a pack'` implies two yellow bell peppers and two red bell peppers.

**Approach**:

1. Replace `x` units ordered with `2x` units each of '`Red Bell Pepper per pc`' and '`Yellow Bell Pepper per pc`'.

In [23]:
# manually handle bell peppers

for index, row in cleaned_df.iterrows():
    if row["item"] == 'Red & Yellow Bell Pepper 2 each in a pack':

        # add red bell peppers
        cleaned_df = cleaned_df.append({"item": "Red Bell Pepper per pc",
                                        "num_units": row["num_units"]*2,
                                        "qty_per_packet": 1,
                                        "total_qty": row["num_units"]*2,
                                        "unit": "pcs"}, ignore_index=True)

        # add yellow bell peppers
        cleaned_df = cleaned_df.append({"item": "Yellow Bell Pepper per pc",
                                        "num_units": row["num_units"]*2,
                                        "qty_per_packet": 1,
                                        "total_qty": row["num_units"]*2,
                                        "unit": "pcs"}, ignore_index=True)

        # remove packet entry
        cleaned_df.drop(index=index, inplace=True)

## Save to Disk!

Save file to disk as `listified-filename.csv`

In [24]:
save_pth = OUTPUT_DIR/FILE.stem
os.makedirs(save_pth, exist_ok=True)

In [25]:
save_pth = save_pth/('listified-' + str(FILE.stem) + '.csv')

In [26]:
cleaned_df.to_csv(save_pth, index=False)

In [27]:
cleaned_df

,item,num_units,qty_per_packet,total_qty,unit
0,Apples Royal Gala Per Kg,22,1,22,kgs
1,Apples Washington Per Kg,18,1,18,kgs
2,Arbi Leaves Pack of 5 leaves,33,5,165,kgs
3,Arbi/ Colocasia 500gms,23,500,11500,gms
4,Baby Corn 300 gms,25,300,7500,gms
...,...,...,...,...,...
57,Tondli 300 gms,40,300,12000,gms
58,VNR Guava Per Kg 3 Pcs,23,3,69,pcs
59,Yellow Zucchini Per Pc,24,1,24,pcs
60,Red Bell Pepper per pc,62,1,62,pcs


In [37]:
# all_costs['Red Bell Pepper per pc'] = 0
# all_costs['Yellow Bell Pepper per pc'] = 0

In [38]:
# cleaned_df['price_per_unit'] = cleaned_df.item.apply(lambda x: all_costs[x])

In [36]:
# cleaned_df

In [39]:
# cleaned_df['total'] = cleaned_df.num_units * cleaned_df.price_per_unit

In [40]:
# cleaned_df.total.sum()